In [26]:
import io
import uuid
from xml.etree import ElementTree
import requests
import pymarc
import scipy.stats as st
from matplotlib import colors
import numpy as np
import datetime
import csv
import matplotlib.gridspec as gridspec




from rdflib import Graph, ConjunctiveGraph, Namespace, OWL, Literal, URIRef, BNode, XSD, RDFS, RDF
from rdfalchemy import rdfSubject, rdfSingle, rdfMultiple
from SPARQLWrapper import SPARQLWrapper, JSON


import networkx as nx
from cdlib import algorithms
import matplotlib.pyplot as plt
from cdlib import evaluation

from cdlib.algorithms import louvain
from cdlib import algorithms, viz



bio = Namespace("http://purl.org/vocab/bio/0.1/")
schema = Namespace('http://schema.org/')
sem = Namespace('http://semanticweb.cs.vu.nl/2009/11/sem/')
pnv = Namespace('https://w3id.org/pnv#')

In [27]:
# Create a Graph
g = Graph()

# Parse in the RDF (.ttl) file 
g.parse("./data/all-schema-ecartico.ttl.txt")

# Loop through each triple in the graph (subj, pred, obj)
for subj, pred, obj in g:
    # Check if there is at least one triple in the Graph
    if (subj, pred, obj) not in g:
       raise Exception("Something went wrong!")

# Print the number of "triples" in the Graph
print(f"Graph g has {len(g)} statements.")

Graph g has 34770 statements.


<h1> Attribute Queries </h1>

In [29]:
sparql = SPARQLWrapper("https://sparql.goldenagents.org/sparql")
sparql.setQuery (""" PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX schema: <http://schema.org/>

SELECT * WHERE {

{
  GRAPH <https://data.goldenagents.org/datasets/u692bc364e9d7fa97b3510c6c0c8f2bb9a0e5123b/ecartico_20211014> {
    ?person a schema:Person ;
        schema:birthDate ?birthDate .
    }
  }
  UNION {
    GRAPH <https://data.goldenagents.org/datasets/u692bc364e9d7fa97b3510c6c0c8f2bb9a0e5123b/ecartico_20211014> {
      ?person a schema:Person ;
         schema:birthDate [ a schema:StructuredValue ;
                            rdf:value ?birthDate ] .
      }
    } 
} 
"""
)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()


In [30]:
sparql.setQuery ("""

PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX schema: <http://schema.org/>
PREFIX ecartico: <http://www.vondel.humanities.uva.nl/ecartico/lod/vocab/#>

SELECT * WHERE {

  GRAPH <https://data.goldenagents.org/datasets/u692bc364e9d7fa97b3510c6c0c8f2bb9a0e5123b/ecartico_20211014> {
    ?person a schema:Person ;
        ecartico:religion ?religionName .
  }
 
} 
""")

sparql.setReturnFormat(JSON)
results = sparql.query().convert()

In [33]:
sparql.setQuery ("""

PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX schema: <http://schema.org/>

SELECT * WHERE {

  GRAPH <https://data.goldenagents.org/datasets/u692bc364e9d7fa97b3510c6c0c8f2bb9a0e5123b/ecartico_20211014> {
    ?person a schema:Person ;
        schema:birthPlace ?birthPlace .
   
    ?birthPlace a schema:Place ;
        schema:name ?birthPlaceName .
  }
 
}
""")

sparql.setReturnFormat(JSON)
results = sparql.query().convert()



In [35]:
sparql.setQuery ("""
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX schema: <http://schema.org/>

SELECT * WHERE {

  GRAPH <https://data.goldenagents.org/datasets/u692bc364e9d7fa97b3510c6c0c8f2bb9a0e5123b/ecartico_20211014> {
    ?person a schema:Person ;
        schema:hasOccupation [ a schema:Role ;
                               schema:hasOccupation ?occupation ] .
   
    ?occupation a schema:Occupation ;
        schema:name ?occupationName .
   
    FILTER(LANG(?occupationName) = 'en')
   
  }
 
}
""")

sparql.setReturnFormat(JSON)
results = sparql.query().convert()




In [37]:
sparql.setQuery ("""
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX schema: <http://schema.org/>

SELECT * WHERE {

  GRAPH <https://data.goldenagents.org/datasets/u692bc364e9d7fa97b3510c6c0c8f2bb9a0e5123b/ecartico_20211014> {
    ?person a schema:Person .
    
    OPTIONAL { ?person schema:parent ?parent . }
    OPTIONAL { ?person schema:children ?child . }
    
    }      
  
}
""")

sparql.setReturnFormat(JSON)
results = sparql.query().convert()



In [42]:
sparql.setQuery ("""

PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX schema: <http://schema.org/>
PREFIX ecartico: <http://www.vondel.humanities.uva.nl/ecartico/lod/vocab/#>

SELECT * WHERE {

  GRAPH <https://data.goldenagents.org/datasets/u692bc364e9d7fa97b3510c6c0c8f2bb9a0e5123b/ecartico_20211014> {
    ?person a schema:Person ;
  		  ?relation [ a schema:Role ;
                    ?relation ?relatedPerson ] .
      
    ?relatedPerson a schema:Person .
  		
    ?relation rdfs:subPropertyOf ecartico:hasRelationWith .
    
    }      
  
}
""")

sparql.setReturnFormat(JSON)
results = sparql.query().convert()



In [43]:
print(len(results["results"]["bindings"]))
for result in results["results"]["bindings"]:
    print(result)


0
